<a href="https://colab.research.google.com/github/Shubhangimahato/Dairy-DigiD-Artificial-Intelligence-Powered-Facial-Recognition/blob/main/Dairy_DigiD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available")
    device = torch.device("cpu")

print(f"Using device: {device}")


In [ ]:
!pip show pyyaml


In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# !nvidia-smi
!nvcc --version #10.1
!python --version # 3.7.6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())


# Loading Detectron 2

In [ ]:
!python -m pip install "git+https://github.com/facebookresearch/detectron2.git"


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode


# Loading Dataset

In [ ]:
import os
import json
from detectron2.structures import BoxMode

# Define the root directories for both '28 May' and '24 June'
root_dirs = {
    '28 May': '/content/drive/My Drive/28 May_00101/28 May/',
    '24 June': '/content/drive/My Drive/24 June/'
}

# Define the subdirectories to include for both '28 May' and '24 June'
subdirs_to_include = {
    '28 May': ['28 May_001', '28 May_002'],
    '24 June': ['24_june_001', '24_june_002', '24_june_003', '24_june_004']
}

# Initialize lists to collect all images and annotations
all_annotations = []

# Loop through the root directories and their specified subdirectories
for root_dir_name, subdirs in subdirs_to_include.items():
    root_dir = root_dirs[root_dir_name]
    for subdir_name in subdirs:
        subdir_path = os.path.join(root_dir, subdir_name)
        annotation_folder = os.path.join(subdir_path, 'annotations')
        images_root_folder = os.path.join(subdir_path, 'images')

        # Load all JSON annotation files from the annotation folder
        for annotation_file in os.listdir(annotation_folder):
            if annotation_file.endswith('.json'):
                json_path = os.path.join(annotation_folder, annotation_file)
                with open(json_path, 'r') as f:
                    annotations = json.load(f)
                    for annotation in annotations['annotations']:
                        annotation['image_id'] = f"{subdir_name}_{annotation['image_id']}"
                    for image_info in annotations['images']:
                        image_info['id'] = f"{subdir_name}_{image_info['id']}"

                        # Remove redundant folder name from 'file_name' if present
                        if image_info['file_name'].startswith(f"{root_dir_name}/"):
                            image_info['file_name'] = image_info['file_name'][len(root_dir_name) + 1:]

                        # Now combine with 'images_root_folder'
                        image_info['file_name'] = os.path.join(images_root_folder, image_info['file_name'])

                    all_annotations.append({
                        'images': annotations['images'],
                        'annotations': annotations['annotations']
                    })

# Now create the dataset dictionary
def custom_cattle_dataset():
    dataset_dicts = []
    for annotation_group in all_annotations:
        images_info_map = {img_info['id']: img_info for img_info in annotation_group['images']}
        for annotation in annotation_group['annotations']:
            record = {}
            image_info = images_info_map.get(annotation['image_id'])
            if image_info is None:
                print(f"Image info not found for image ID: {annotation['image_id']}")
                continue
            record['file_name'] = image_info['file_name']
            record['image_id'] = image_info['id']
            record['height'] = image_info['height']
            record['width'] = image_info['width']
            # Determine category based on file path
            if "2 Years old or less" in record["file_name"] or "Heifer" in record["file_name"]:
                category_id = 0
            elif "Dry Cows" in record["file_name"] or "dry cows" in record["file_name"]:
                category_id = 1
            elif "Mature Milking Cow" in record["file_name"]:
                category_id = 2
            elif "Pregnant" in record["file_name"]:
                category_id = 3
            else:
                category_id = 999  # Undefined category (optional)
            obj = {
                'bbox': annotation['bbox'],
                'bbox_mode': BoxMode.XYWH_ABS,
                'category_id': category_id,
                'segmentation': annotation.get('segmentation', []),
                'keypoints': annotation.get('keypoints', []),
                'iscrowd': annotation.get('iscrowd', 0),
            }
            record['annotations'] = [obj]
            dataset_dicts.append(record)
    return dataset_dicts

# Generate the dataset dictionary
dataset_dicts = custom_cattle_dataset()
print("Dataset Dicts Sample:", dataset_dicts[:5])


In [ ]:
def count_images_by_category(dataset_dicts):
    category_count = {
        "Pregnant": 0,
        "Mature Milking Cow": 0,
        "Dry Cows": 0,
        "2 Years old or less": 0
    }
    category_mapping = {
        0: "2 Years old or less",
        1: "Dry Cows",
        2: "Mature Milking Cow",
        3: "Pregnant",
        999: "Undefined"
    }
    for record in dataset_dicts:
        if record['annotations']:
            category_id = record['annotations'][0]['category_id']
            category_label = category_mapping.get(category_id, "Undefined")
            if category_label in category_count:
                category_count[category_label] += 1
    return category_count

category_image_count = count_images_by_category(dataset_dicts)
print(category_image_count)


In [ ]:
# Define the keypoint flip map using numerical points
keypoint_flip_map = [
    (1, 5),   # Left Eye extreme right <-> Right Eye extreme left
    (2, 6),   # Left Eye extreme left <-> Right Eye extreme righta
    (3, 7),   # Left Eye extreme top <-> Right Eye extreme top
    (4, 8),   # Left Eye extreme bottom <-> Right Eye extreme bottom
    (9, 14),  # Left Ear extreme top right <-> Right Ear extreme top left
    (10, 15), # Left Ear extreme left <-> Right Ear extreme right
    (11, 16), # Left Ear extreme top mid <-> Right Ear extreme top mid
    (12, 17), # Left Ear extreme bottom mid <-> Right Ear extreme bottom mid
    (13, 18), # Left Ear extreme bottom right <-> Right Ear extreme bottom left
    (19, 21), # Muzzle Top left <-> Muzzle Top right
    (20, 20), # Muzzle Top mid <-> Muzzle Top mid (no change)
    (21, 19), # Muzzle Top right <-> Muzzle Top left
    (22, 24), # Muzzle Bottom right <-> Muzzle Bottom left
    (23, 23), # Muzzle Bottom mid <-> Muzzle Bottom mid (no change)
    (24, 22), # Muzzle Bottom left <-> Muzzle Bottom right
    (28, 30), # Head left side <-> Head right side
    (29, 29), # Head extreme top <-> Head extreme top (no change)
    (30, 28), # Head right side <-> Head left side
]

# Define the keypoint names for 30 points
keypoint_names = [
    'kp1', 'kp2', 'kp3', 'kp4', 'kp5', 'kp6', 'kp7', 'kp8', 'kp9', 'kp10',
    'kp11', 'kp12', 'kp13', 'kp14', 'kp15', 'kp16', 'kp17', 'kp18', 'kp19', 'kp20',
    'kp21', 'kp22', 'kp23', 'kp24', 'kp25', 'kp26', 'kp27', 'kp28', 'kp29', 'kp30'
]

In [ ]:

import random
from detectron2.data import DatasetCatalog, MetadataCatalog

# Function to split dataset into train, val, and test sets
def split_dataset(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    """
    Splits the dataset into training, validation, and test sets.
    """
    assert train_ratio + val_ratio + test_ratio == 1, "Ratios must sum up to 1!"

    # Set random seed for reproducibility
    random.seed(seed)

    # Shuffle dataset
    shuffled_dataset = random.sample(dataset, len(dataset))

    # Compute split indices
    total_size = len(dataset)
    train_size = int(total_size * train_ratio)
    val_size = int(total_size * val_ratio)

    # Split dataset
    train_set = shuffled_dataset[:train_size]
    val_set = shuffled_dataset[train_size:train_size + val_size]
    test_set = shuffled_dataset[train_size + val_size:]

    return train_set, val_set, test_set

# Ensure your dataset is already loaded in `dataset_dicts`
train_set, val_set, test_set = split_dataset(dataset_dicts)

# Print dataset sizes
print(f"Train Set Size: {len(train_set)}")
print(f"Validation Set Size: {len(val_set)}")
print(f"Test Set Size: {len(test_set)}")

# Define functions to return datasets
def get_cows_train():
    return train_set

def get_cows_val():
    return val_set

def get_cows_test():
    return test_set

# Register datasets
DatasetCatalog.register("cows_train", get_cows_train)
DatasetCatalog.register("cows_val", get_cows_val)
DatasetCatalog.register("cows_test", get_cows_test)

# Define common metadata
metadata_info = {
    "keypoint_flip_map": keypoint_flip_map,
    "keypoint_names": keypoint_names,
    "thing_classes": ["Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"],
    "evaluator_type": "coco"
}

# Apply metadata to all datasets
for dataset_name in ["cows_train", "cows_val", "cows_test"]:
    MetadataCatalog.get(dataset_name).keypoint_flip_map = metadata_info["keypoint_flip_map"]
    MetadataCatalog.get(dataset_name).keypoint_names = metadata_info["keypoint_names"]
    MetadataCatalog.get(dataset_name).thing_classes = metadata_info["thing_classes"]
    MetadataCatalog.get(dataset_name).evaluator_type = metadata_info["evaluator_type"]

print("Datasets registered successfully with metadata!")



# K Fold Validation

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Define the model creation function
def create_model(learning_rate, num_filters, kernel_size):
    model = Sequential([
        Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # Adjust based on the number of classes
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# K-Fold Cross-Validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

fold = 1
fold_accuracies = []
fold_f1_scores = []

# Convert data to numpy arrays for KFold
X = train_images_processed
y = train_labels_processed

for train_index, val_index in kf.split(X):
    print(f"Training Fold {fold}/{k_folds}...")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create and train the model
    model = create_model(learning_rate=0.001, num_filters=32, kernel_size=3)
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0,
                        validation_data=(X_val, y_val))

    # Evaluate the model on the validation set
    y_val_pred = model.predict(X_val)
    y_val_pred_classes = np.argmax(y_val_pred, axis=1)

    accuracy = accuracy_score(y_val, y_val_pred_classes)
    f1 = f1_score(y_val, y_val_pred_classes, average='weighted')
    fold_accuracies.append(accuracy)
    fold_f1_scores.append(f1)

    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")
    fold += 1

# Statistical Analysis
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)
mean_f1 = np.mean(fold_f1_scores)
std_f1 = np.std(fold_f1_scores)

print("\nCross-Validation Results:")
print(f"Mean Accuracy: {mean_accuracy:.4f}, Std Accuracy: {std_accuracy:.4f}")
print(f"Mean F1-Score: {mean_f1:.4f}, Std F1-Score: {std_f1:.4f}")


In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, roc_curve
import matplotlib.pyplot as plt

# Define the model creation function
def create_model(learning_rate, num_filters, kernel_size):
    model = Sequential([
        Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # Adjust based on number of classes
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# K-Fold Cross-Validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

fold = 1
fold_accuracies = []
fold_f1_scores = []
fold_auc_scores = []

# Convert data to numpy arrays for KFold
X = train_images_processed
y = train_labels_processed

for train_index, val_index in kf.split(X):
    print(f"Training Fold {fold}/{k_folds}...")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create and train the model
    model = create_model(learning_rate=0.001, num_filters=32, kernel_size=3)
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0,
                        validation_data=(X_val, y_val))

    # Evaluate the model on the validation set
    y_val_pred = model.predict(X_val)
    y_val_pred_classes = np.argmax(y_val_pred, axis=1)

    accuracy = accuracy_score(y_val, y_val_pred_classes)
    f1 = f1_score(y_val, y_val_pred_classes, average='weighted')
    auc = roc_auc_score(y_val, y_val_pred, multi_class='ovr', average='weighted')

    fold_accuracies.append(accuracy)
    fold_f1_scores.append(f1)
    fold_auc_scores.append(auc)

    print(f"Fold {fold} - Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, AUC-ROC: {auc:.4f}")
    fold += 1

# Statistical Analysis (Validation Performance)
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)
mean_f1 = np.mean(fold_f1_scores)
std_f1 = np.std(fold_f1_scores)
mean_auc = np.mean(fold_auc_scores)
std_auc = np.std(fold_auc_scores)

print("\nValidation Cross-Validation Results:")
print(f"Mean Accuracy: {mean_accuracy:.4f}, Std Accuracy: {std_accuracy:.4f}")
print(f"Mean F1-Score: {mean_f1:.4f}, Std F1-Score: {std_f1:.4f}")
print(f"Mean AUC-ROC: {mean_auc:.4f}, Std AUC-ROC: {std_auc:.4f}")

# ----------------- Final Model Training and Test Evaluation -----------------

print("\nTraining Final Model on Full Training Data...")
final_model = create_model(learning_rate=0.001, num_filters=32, kernel_size=3)
final_model.fit(X, y, epochs=10, batch_size=64, verbose=1)

# Evaluate on test data
print("\nEvaluating on Test Set...")
y_test_pred = final_model.predict(test_images_processed)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

test_accuracy = accuracy_score(test_labels_processed, y_test_pred_classes)
test_f1 = f1_score(test_labels_processed, y_test_pred_classes, average='weighted')
test_auc = roc_auc_score(test_labels_processed, y_test_pred, multi_class='ovr', average='weighted')

print("\nTest Set Results:")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1-Score: {test_f1:.4f}")
print(f"Test AUC-ROC: {test_auc:.4f}")

# Plot ROC Curve
plt.figure(figsize=(8, 6))
for i in range(y_test_pred.shape[1]):
    fpr, tpr, _ = roc_curve(test_labels_processed == i, y_test_pred[:, i])
    plt.plot(fpr, tpr, label=f'Class {i}')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Test Set')
plt.legend()
plt.show()


## Hyperparameter Tuning

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def create_model(learning_rate, num_filters, kernel_size):
    model = Sequential([
        Conv2D(num_filters, (kernel_size, kernel_size), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(num_filters * 2, (kernel_size, kernel_size), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(24, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

learning_rates = [0.001, 0.0001, 0.01]
num_filters = [32, 64, 128]
kernel_sizes = [3, 5]

best_accuracy = 0
best_params = {}

for lr in learning_rates:
    for nf in num_filters:
        for ks in kernel_sizes:
            model = create_model(lr, nf, ks)
            history = model.fit(train_images_processed, train_labels_processed, epochs=5, batch_size=64,
                                validation_data=(val_images_processed, val_labels_processed), verbose=0)
            val_accuracy = history.history['val_accuracy'][-1]
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {'learning_rate': lr, 'num_filters': nf, 'kernel_size': ks}

print("Best Hyperparameters from Random Search:", best_params)
print("Best Validation Accuracy:", best_accuracy)


In [ ]:
learning_rate = 0.001  # Fixed as it performs well
num_filters_range = [64, 96, 128]
kernel_size_range = [3, 4, 5]

best_accuracy = 0
best_params = {}

for nf in num_filters_range:
    for ks in kernel_size_range:
        model = create_model(learning_rate, nf, ks)
        history = model.fit(train_images_processed, train_labels_processed, epochs=5, batch_size=64,
                            validation_data=(val_images_processed, val_labels_processed), verbose=0)
        val_accuracy = history.history['val_accuracy'][-1]
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = {'num_filters': nf, 'kernel_size': ks}

print("Best Hyperparameters after Refined Grid Search:", best_params)
print("Best Validation Accuracy:", best_accuracy)


# Preprocessing

In [ ]:
from detectron2.modeling import build_model
from detectron2.modeling.meta_arch import GeneralizedRCNN
from detectron2.modeling.roi_heads import StandardROIHeads
from detectron2.config import get_cfg

cfg = get_cfg()

class FocalLossRCNN(GeneralizedRCNN):
    def forward(self, batched_inputs):
        if self.training:
            losses = super().forward(batched_inputs)
            # Modify losses["loss_cls"] with a custom focal loss
            losses["loss_cls"] = focal_loss_function(self.pred_class_logits, self.gt_classes)
            return losses
        else:
            return super().forward(batched_inputs)

# Integrate into Trainer
cfg.MODEL.ROI_HEADS.LOSS = "FocalLossRCNN"


In [ ]:
cfg.MODEL.BOX_LOSS_TYPE = "siou"  # Custom loss integration
cfg.SOLVER.OPTIMIZER = "AdamW"
cfg.SOLVER.BASE_LR = 0.0001  # Adjust learning rate for AdamW


In [ ]:
from detectron2.data.samplers import RepeatFactorTrainingSampler

def build_train_loader(cfg):
    dataset_dicts = DatasetCatalog.get(cfg.DATASETS.TRAIN[0])
    repeat_factors = RepeatFactorTrainingSampler.repeat_factors_from_category_frequency(
        dataset_dicts, repeat_thresh=0.1
    )
    return build_detection_train_loader(cfg, sampler=RepeatFactorTrainingSampler(repeat_factors))


In [ ]:
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.MODEL.BACKBONE.NAME = "build_resnet_fpn_backbone"
cfg.MODEL.RESNETS.DEPTH = 101
cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]


In [ ]:
cfg.MODEL.FPN.NORM = "GN"  # Group Normalization for FPN
cfg.MODEL.FPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]  # Include lower and higher layers


In [ ]:
cfg.MODEL.BACKBONE.ATTENTION_TYPE = "CoordinateAttention"  # Requires custom implementation


In [ ]:
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[32, 64, 128, 256, 512]]  # Adjust based on dataset
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]  # Common ratios


In [ ]:
#Without Argumentation

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from detectron2.engine import DefaultTrainer
from detectron2.engine import HookBase
from detectron2.config import get_cfg
from detectron2 import model_zoo
import numpy as np
from detectron2.data import MetadataCatalog, DatasetCatalog

# AugmentedTrainer and custom_mapper should be defined before this cell.
# EarlyStopping class should also be defined if you're using early stopping.

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cows_train_filtered",)
cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 30
cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((30, 1), dtype=float).tolist()
cfg.SOLVER.STEPS = [3000, 3800]
cfg.SOLVER.GAMMA = 0.1

# Define the keypoint flip map using numerical points
keypoint_flip_map = [
    (1, 5), (2, 6), (3, 7), (4, 8), (9, 14), (10, 15), (11, 16), (12, 17),
    (13, 18), (19, 21), (20, 20), (21, 19), (22, 24), (23, 23), (24, 22),
    (28, 30), (29, 29), (30, 28)
]

# Define the keypoint names for 30 points
keypoint_names = [f'kp{i}' for i in range(1, 31)]

# Keypoint metadata setup
MetadataCatalog.get("cows_train_filtered").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("cows_train_filtered").keypoint_names = keypoint_names
MetadataCatalog.get("cows_train_filtered").thing_classes = [
    "Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"
]
MetadataCatalog.get("cows_train_filtered").evaluator_type = "coco"

# Function to filter out missing files
def check_alternative_extensions(file_path):
    """Check if a file exists with a different extension (case mismatch fix)."""
    base, _ = os.path.splitext(file_path)
    extensions = [".jpg", ".jpeg", ".JPG", ".JPEG", ".png"]
    for ext in extensions:
        new_path = base + ext
        if os.path.exists(new_path):
            return new_path  # Return the corrected path
    return None  # No alternative found

def filter_missing_files(dataset_dicts):
    """Filter dataset to exclude missing files or correct paths with case mismatch."""
    filtered_dataset = []
    for record in dataset_dicts:
        file_path = record["file_name"]
        if os.path.exists(file_path):
            filtered_dataset.append(record)
        else:
            corrected_path = check_alternative_extensions(file_path)
            if corrected_path:
                record["file_name"] = corrected_path  # Update file path
                filtered_dataset.append(record)
    return filtered_dataset

# Register the filtered dataset
dataset_dicts = DatasetCatalog.get("cows_train")
filtered_dataset_dicts = filter_missing_files(dataset_dicts)

# Dataset registration function
def get_filtered_cows_train():
    return filtered_dataset_dicts

# Re-registering the dataset
if "cows_train_filtered" in DatasetCatalog.list():
    DatasetCatalog.remove("cows_train_filtered")
    MetadataCatalog.remove("cows_train_filtered")

DatasetCatalog.register("cows_train_filtered", get_filtered_cows_train)
MetadataCatalog.get("cows_train_filtered").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("cows_train_filtered").keypoint_names = keypoint_names
MetadataCatalog.get("cows_train_filtered").thing_classes = [
    "Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"
]
MetadataCatalog.get("cows_train_filtered").evaluator_type = "coco"

# Set output directory
cfg.OUTPUT_DIR = "./my_custom_output_dir"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)
trainer.build_train_loader = build_train_loader
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# With Argumentation
import os
import cv2
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
import numpy as np
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import build_detection_train_loader

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# AugmentedTrainer and custom_mapper should be defined before this cell.
# EarlyStopping class should also be defined if you're using early stopping.

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cows_train_filtered",)
cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 30
cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((30, 1), dtype=float).tolist()
cfg.SOLVER.STEPS = [3000, 3800]
cfg.SOLVER.GAMMA = 0.1

# Define the keypoint flip map using numerical points
keypoint_flip_map = [
    (1, 5), (2, 6), (3, 7), (4, 8), (9, 14), (10, 15), (11, 16), (12, 17),
    (13, 18), (19, 21), (20, 20), (21, 19), (22, 24), (23, 23), (24, 22),
    (28, 30), (29, 29), (30, 28)
]

# Define the keypoint names for 30 points
keypoint_names = [f'kp{i}' for i in range(1, 31)]

# Keypoint metadata setup
MetadataCatalog.get("cows_train_filtered").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("cows_train_filtered").keypoint_names = keypoint_names
MetadataCatalog.get("cows_train_filtered").thing_classes = [
    "Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"
]
MetadataCatalog.get("cows_train_filtered").evaluator_type = "coco"

# Function to filter out missing files
def check_alternative_extensions(file_path):
    """Check if a file exists with a different extension (case mismatch fix)."""
    base, _ = os.path.splitext(file_path)
    extensions = [".jpg", ".jpeg", ".JPG", ".JPEG", ".png"]
    for ext in extensions:
        new_path = base + ext
        if os.path.exists(new_path):
            return new_path  # Return the corrected path
    return None  # No alternative found

def filter_missing_files(dataset_dicts):
    """Filter dataset to exclude missing files or correct paths with case mismatch."""
    filtered_dataset = []
    for record in dataset_dicts:
        file_path = record["file_name"]
        if os.path.exists(file_path):
            filtered_dataset.append(record)
        else:
            corrected_path = check_alternative_extensions(file_path)
            if corrected_path:
                record["file_name"] = corrected_path  # Update file path
                filtered_dataset.append(record)
    return filtered_dataset

# Register the filtered dataset
dataset_dicts = DatasetCatalog.get("cows_train")
filtered_dataset_dicts = filter_missing_files(dataset_dicts)

# Dataset registration function
def get_filtered_cows_train():
    return filtered_dataset_dicts

# Re-registering the dataset
if "cows_train_filtered" in DatasetCatalog.list():
    DatasetCatalog.remove("cows_train_filtered")
    MetadataCatalog.remove("cows_train_filtered")

DatasetCatalog.register("cows_train_filtered", get_filtered_cows_train)
MetadataCatalog.get("cows_train_filtered").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("cows_train_filtered").keypoint_names = keypoint_names
MetadataCatalog.get("cows_train_filtered").thing_classes = [
    "Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"
]
MetadataCatalog.get("cows_train_filtered").evaluator_type = "coco"

# Basic augmentations for all classes
def standard_augmentations(image):
    transform_list = [
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.MotionBlur(p=0.3),
        A.Rotate(limit=10, p=0.7),
        A.RandomCrop(height=int(image.shape[0] * 0.85), width=int(image.shape[1] * 0.85), p=0.7),
        ToTensorV2()
    ]
    augmented = A.Compose(transform_list)(image=image)
    return augmented["image"].permute(1, 2, 0).numpy()

# Additional augmentations for "Pregnant" class
def pregnant_extra_augmentations(image):
    transform_list = [
        A.GaussNoise(var_limit=50.0, p=0.5),
        A.RandomBrightnessContrast(p=0.7),
        A.MotionBlur(p=0.5),
        ToTensorV2()
    ]
    augmented = A.Compose(transform_list)(image=image)
    return augmented["image"].permute(1, 2, 0).numpy()

# Custom Mapper
def custom_mapper(dataset_dict):
    dataset_dict = dataset_dict.copy()
    image = cv2.imread(dataset_dict["file_name"])
    if image is None:
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    category_id = dataset_dict["annotations"][0]["category_id"]
    image = standard_augmentations(image)
    if category_id == 3:  # Apply additional augmentations for Pregnant class
        image = pregnant_extra_augmentations(image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1))
    return dataset_dict

# Define Custom Trainer
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

# Set output directory
cfg.OUTPUT_DIR = "./my_custom_output_dir"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Specify source and new destination paths
source_path = "./my_custom_output_dir/model_final.pth"
destination_path = "/content/drive/My Drive/model1_final.pth"  # Change this to the desired path in your Google Drive

# Copy the model file using shutil
shutil.copy(source_path, destination_path)

print(f"Model copied to {destination_path}")


In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import build_detection_test_loader, MetadataCatalog, DatasetCatalog
import numpy as np

# Load the config file
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TEST = ("cows_val_filtered",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "./my_custom_output_dir/model_final.pth"  # Path to the trained model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 30
cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((30, 1), dtype=float).tolist()
cfg.OUTPUT_DIR = "./my_custom_output_dir"

# Define keypoint metadata
keypoint_flip_map = [
    (1, 5), (2, 6), (3, 7), (4, 8), (9, 14), (10, 15), (11, 16), (12, 17),
    (13, 18), (19, 21), (20, 20), (21, 19), (22, 24), (23, 23), (24, 22),
    (28, 30), (29, 29), (30, 28)
]
keypoint_names = [f'kp{i}' for i in range(1, 31)]

# Register validation dataset
dataset_dicts = DatasetCatalog.get("cows_val")

# Ensure dataset paths are valid
def filter_missing_files(dataset_dicts):
    filtered_dataset = []
    for record in dataset_dicts:
        if os.path.exists(record["file_name"]):
            filtered_dataset.append(record)
    return filtered_dataset

filtered_dataset_dicts = filter_missing_files(dataset_dicts)

if "cows_val_filtered" in DatasetCatalog.list():
    DatasetCatalog.remove("cows_val_filtered")
    MetadataCatalog.remove("cows_val_filtered")

DatasetCatalog.register("cows_val_filtered", lambda: filtered_dataset_dicts)
MetadataCatalog.get("cows_val_filtered").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("cows_val_filtered").keypoint_names = keypoint_names
MetadataCatalog.get("cows_val_filtered").thing_classes = [
    "Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"
]
MetadataCatalog.get("cows_val_filtered").evaluator_type = "coco"

# Run evaluation
evaluator = COCOEvaluator("cows_val_filtered", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "cows_val_filtered")
inference_on_dataset(DefaultTrainer.build_model(cfg), val_loader, evaluator)

# Results

**AUC-ROC** Curve

In [ ]:

from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import numpy as np

categories = ["Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"]

# Load model weights and set up predictor
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.DATASETS.TEST = ("cows_train_filtered",)
predictor = DefaultPredictor(cfg)

actual_labels = []
predicted_probs = []

def extract_ground_truth_from_filepath(file_path):
    if "Young Cows" in file_path or "2 Years old or less" in file_path or "Heifer" in file_path:
        return 0  # Category index
    elif "Dry Cows" in file_path:
        return 1
    elif "Mature Milking Cow" in file_path:
        return 2
    elif "Pregnant" in file_path:
        return 3
    else:
        return None

# Process images for evaluation
for root, dirs, files in os.walk('/content/drive/My Drive/test'):
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):
            file_path = os.path.join(root, file)
            image = cv2.imread(file_path)
            if image is None:
                print(f"Error loading image: {file_path}")
                continue

            # Run inference
            outputs = predictor(image)
            probs = outputs["instances"].scores.cpu().numpy() if len(outputs["instances"]) > 0 else []
            classes = outputs["instances"].pred_classes.cpu().numpy() if len(outputs["instances"]) > 0 else []

            ground_truth = extract_ground_truth_from_filepath(file_path)
            if ground_truth is not None:
                actual_labels.append(ground_truth)

                # Assign predicted probabilities
                if len(probs) > 0:
                    prob_vector = np.zeros(len(categories))  # Initialize probabilities for all classes
                    for cls, prob in zip(classes, probs):
                        prob_vector[cls] = prob
                    prob_vector = F.softmax(torch.tensor(prob_vector), dim=0).numpy()  # Normalize with softmax
                    predicted_probs.append(prob_vector)
                else:
                    predicted_probs.append(np.zeros(len(categories)))  # No prediction

# Convert to numpy arrays
actual_labels = np.array(actual_labels)
predicted_probs = np.array(predicted_probs)

# Calculate AUC-ROC
try:
    auc_roc = roc_auc_score(actual_labels, predicted_probs, multi_class="ovr", average="weighted")
    print(f"AUC-ROC: {auc_roc:.4f}")
except ValueError as e:
    print(f"Error calculating AUC-ROC: {e}")


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Calculate ROC curve and AUC for each class
fpr = {}
tpr = {}
roc_auc = {}

for i in range(len(categories)):
    # Binarize the actual labels for the current class
    binary_actual = (actual_labels == i).astype(int)
    fpr[i], tpr[i], _ = roc_curve(binary_actual, predicted_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 7))
for i in range(len(categories)):
    plt.plot(
        fpr[i], tpr[i],
        label=f"Class {categories[i]} (AUC = {roc_auc[i]:.2f})"
    )

# Add random chance line
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")

# Graph styling
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Multi-Class ROC Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from sklearn.metrics import confusion_matrix, classification_report

categories = ["Young Cows", "Dry Cows", "Mature Milking Cow", "Pregnant"]

# Load model weights and set up predictor
cfg.MODEL.WEIGHTS = "/content/drive/My Drive/model1_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
cfg.DATASETS.TEST = ("cows_test",)
predictor = DefaultPredictor(cfg)

predicted_labels = []
actual_labels = []

def extract_ground_truth_from_filepath(file_path):
    # Returns category label based on file path
    if "Young Cows" in file_path or "2 Years old or less" in file_path or "Heifer" in file_path:
        return "Young Cows"
    elif "Dry Cows" in file_path:
        return "Dry Cows"
    elif "Mature Milking Cow" in file_path:
        return "Mature Milking Cow"
    elif "Pregnant" in file_path:
        return "Pregnant"
    else:
        return None  # Return None for undefined categories

def process_images_for_evaluation(folder_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith((".jpg", ".jpeg", ".png")):
                file_path = os.path.join(root, file)
                image = cv2.imread(file_path)
                if image is None:
                    print(f"Error loading image: {file_path}")
                    continue

                # Run inference
                outputs = predictor(image)
                metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])
                visualizer = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.2)
                out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))

                # Save visualized output
                output_image = out.get_image()[:, :, ::-1]
                relative_path = os.path.relpath(root, folder_path)
                output_subfolder = os.path.join(output_folder, relative_path)
                os.makedirs(output_subfolder, exist_ok=True)
                output_image_path = os.path.join(output_subfolder, file)
                cv2.imwrite(output_image_path, output_image)

                # Skip if no predictions are made
                if len(outputs["instances"].pred_classes) == 0:
                    print(f"No prediction for image: {file_path}")
                    continue

                # Map predictions and ground truth labels
                predicted_class_idx = outputs["instances"].pred_classes[0].item()
                predicted_class = categories[predicted_class_idx]
                actual_class = extract_ground_truth_from_filepath(file_path)

                # Append only valid actual labels
                if actual_class is not None:
                    predicted_labels.append(predicted_class)
                    actual_labels.append(actual_class)

# Run evaluation
input_folder = '/content/drive/My Drive/test'
output_folder = '/content/drive/My Drive/test_results'
process_images_for_evaluation(input_folder, output_folder)

# Print evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(actual_labels, predicted_labels, labels=categories))
print("\nClassification Report:")
print(classification_report(actual_labels, predicted_labels, target_names=categories))


In [ ]:
def ensemble_predictions(image, predictors):
    results = []
    for predictor in predictors:
        outputs = predictor(image)
        results.append(outputs["instances"].pred_classes.tolist())
    # Aggregate predictions using majority voting
    aggregated = Counter([item for sublist in results for item in sublist])
    return aggregated.most_common(1)[0][0]  # Return the most common class


In [ ]:

# Generate classification report
report = classification_report(actual_labels, predicted_labels, target_names=categories, output_dict=True)

# Extract metrics for visualization
precision = [report[cls]['precision'] for cls in categories]
recall = [report[cls]['recall'] for cls in categories]
f1_score = [report[cls]['f1-score'] for cls in categories]

# Radar Chart
def plot_radar_chart(categories, precision, recall, f1_score):
    metrics = ['Precision', 'Recall', 'F1-Score']
    values = [precision, recall, f1_score]
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]

    values = [list(metric) + [metric[0]] for metric in values]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    for idx, value in enumerate(values):
        ax.plot(angles, value, label=metrics[idx], linewidth=2)
        ax.fill(angles, value, alpha=0.25)

    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_title("Classification Metrics Radar Chart")
    ax.legend(loc="upper right", bbox_to_anchor=(1.2, 1.2))
    plt.show()

plot_radar_chart(categories, precision, recall, f1_score)

# Bar Chart for Comparison
def plot_bar_chart(categories, precision, recall, f1_score):
    x = np.arange(len(categories))
    width = 0.25

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(x - width, precision, width, label="Precision", color="skyblue")
    ax.bar(x, recall, width, label="Recall", color="lightgreen")
    ax.bar(x + width, f1_score, width, label="F1-Score", color="salmon")

    ax.set_xlabel("Categories")
    ax.set_ylabel("Scores")
    ax.set_title("Classification Metrics Comparison")
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.legend()

    plt.tight_layout()
    plt.show()

plot_bar_chart(categories, precision, recall, f1_score)

In [ ]:
import seaborn as sns
# Generate the confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels, labels=categories)
print("Confusion Matrix:")
print(conf_matrix)

# Generate the classification report
class_report = classification_report(actual_labels, predicted_labels, target_names=categories, output_dict=True)
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels, target_names=categories))

# Plot Confusion Matrix
def plot_confusion_matrix(conf_matrix, categories):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
    plt.title('Confusion Matrix')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.show()

# Plot Precision, Recall, F1-Score
def plot_classification_report(class_report, categories):
    metrics = ['precision', 'recall', 'f1-score']
    scores = {metric: [class_report[category][metric] for category in categories] for metric in metrics}

    x = np.arange(len(categories))  # label locations
    width = 0.2  # width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))

    rects1 = ax.bar(x - width, scores['precision'], width, label='Precision')
    rects2 = ax.bar(x, scores['recall'], width, label='Recall')
    rects3 = ax.bar(x + width, scores['f1-score'], width, label='F1-Score')

    ax.set_xlabel('Categories')
    ax.set_ylabel('Scores')
    ax.set_title('Classification Metrics by Category')
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.legend()

    fig.tight_layout()
    plt.show()

# Plot confusion matrix
plot_confusion_matrix(conf_matrix, categories)

# Plot classification report metrics
plot_classification_report(class_report, categories)


In [ ]:
import json
import matplotlib.pyplot as plt
import os

# Set the path to the metrics.json file
output_dir = cfg.OUTPUT_DIR
metrics_file = os.path.join(output_dir, "metrics.json")

# Load the metrics from the JSON file
metrics = []
with open(metrics_file, 'r') as f:
    for line in f:
        metrics.append(json.loads(line))

# Extract the iteration values, training loss, and validation loss
iterations = [x['iteration'] for x in metrics if 'total_loss' in x]
train_losses = [x['total_loss'] for x in metrics if 'total_loss' in x]

# Validation loss may have different key, make sure to check the key in your JSON
val_iterations = [x['iteration'] for x in metrics if 'validation_loss' in x]
val_losses = [x['validation_loss'] for x in metrics if 'validation_loss' in x]

# Plot both the training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(iterations, train_losses, label='Training Loss', color='blue')
if len(val_iterations) > 0 and len(val_losses) > 0:
    plt.plot(val_iterations, val_losses, label='Validation Loss', color='red')

plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Iterations')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
output_dir = '/content/drive/MyDrive/detectron2_models'
os.makedirs(output_dir, exist_ok=True)


In [ ]:
import shutil

# Path where your model is saved after training
final_model_path = "./my_custom_output_dir/model_final.pth"

# Target path in Google Drive
saved_model_path = "/content/drive/My Drive/model/model_final.pth"

# Ensure the target directory exists
target_dir = "/content/drive/My Drive/model"
os.makedirs(target_dir, exist_ok=True)

# Copy the model file to Google Drive
shutil.copy(final_model_path, saved_model_path)

print(f"Model saved to {saved_model_path}")

# GRAD Cam

In [ ]:
!pip install grad-cam


In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2 import model_zoo

# Load the trained model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # Adjust based on your classes
cfg.MODEL.WEIGHTS = "/content/drive/My Drive/model1_final.pth"  # Change to your model path
model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)
model.eval()

# Grad-CAM Hook Functions
feature_maps = None
gradients = None

def forward_hook(module, input, output):
    global feature_maps
    feature_maps = output

def backward_hook(module, grad_in, grad_out):
    global gradients
    gradients = grad_out[0]

# Register hooks to a deeper convolutional layer (res4 for better feature visualization)
target_layer = model.backbone.bottom_up.res4
target_layer.register_forward_hook(forward_hook)
target_layer.register_full_backward_hook(backward_hook)

# Function to generate Grad-CAM
def generate_grad_cam(image_path):
    global feature_maps, gradients

    torch.cuda.empty_cache()  # Free up GPU memory

    # Load and preprocess image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image at '{image_path}'. Check the path.")
        return

    img = cv2.resize(img, (512, 512))  # Reduce image size to avoid memory overflow
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert to tensor and send to GPU
    img_tensor = torch.tensor(img).permute(2, 0, 1).float().unsqueeze(0).cuda()

    # Forward pass
    model.zero_grad()
    inputs = [{"image": img_tensor.squeeze()}]
    outputs = model(inputs)[0]  # Extract first output dictionary

    if "instances" not in outputs:
        print("Error: No instances detected in the output.")
        return

    instances = outputs["instances"]
    if len(instances) == 0:
        print("No objects detected in this image.")
        return

    # Compute gradients for the highest scoring class
    scores = instances.scores
    highest_class_idx = scores.argmax()
    scores[highest_class_idx].backward()

    # **Fix tensor mismatch issue**
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])  # Average over height and width
    feature_maps = feature_maps.squeeze(0)  # Remove batch dimension

    # Reshape pooled gradients to match feature maps
    pooled_gradients = pooled_gradients.view(feature_maps.shape[0], 1, 1)

    # Compute Grad-CAM heatmap
    cam = torch.sum(pooled_gradients * feature_maps, dim=0).cpu().detach().numpy()
    cam = np.maximum(cam, 0)  # Apply ReLU
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))  # Resize to match original image size

    # Overlay the heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * cam / cam.max()), cv2.COLORMAP_JET)
    overlayed_img = cv2.addWeighted(img, 0.5, heatmap, 0.5, 0)

    # Display result
    plt.figure(figsize=(8, 8))
    plt.imshow(overlayed_img)
    plt.axis("off")
    plt.title("Grad-CAM Heatmap")
    plt.show()

# Run Grad-CAM on a sample image
generate_grad_cam("/content/drive/My Drive/Rename.JPG")


In [ ]:
import os
import cv2
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import preprocess_image as gradcam_preprocess
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget

# Load the trained Detectron2 model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # Adjust based on your dataset
cfg.MODEL.WEIGHTS = "/content/drive/My Drive/model1_final.pth"  # Change to your trained model path
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

# Initialize model manually for Grad-CAM
model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)
model.eval()

# Register Grad-CAM Hook
feature_maps = None
gradients = None

def forward_hook(module, input, output):
    global feature_maps
    feature_maps = output

def backward_hook(module, grad_in, grad_out):
    global gradients
    gradients = grad_out[0]

# Register hooks to a deeper convolutional layer (res4 for better feature visualization)
target_layer = model.backbone.bottom_up.res4
target_layer.register_forward_hook(forward_hook)
target_layer.register_full_backward_hook(backward_hook)

# Initialize predictor
predictor = DefaultPredictor(cfg)

# Function to get 8 random images from test folder
def get_random_images(folder, num_images=8):
    image_paths = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):  # Check valid image formats
                image_paths.append(os.path.join(root, file))

    if len(image_paths) == 0:
        raise ValueError("❌ ERROR: No images found in test folder!")

    random.shuffle(image_paths)
    return image_paths[:num_images]

# Function to generate Grad-CAM heatmap
def generate_grad_cam(image_path):
    global feature_maps, gradients

    torch.cuda.empty_cache()  # Free up GPU memory

    # Load and preprocess image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image at '{image_path}'. Check the path.")
        return None, None

    img = cv2.resize(img, (512, 512))  # Resize to avoid memory overflow
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert to tensor and send to GPU
    img_tensor = torch.tensor(img).permute(2, 0, 1).float().unsqueeze(0).cuda()

    # Forward pass
    model.zero_grad()
    inputs = [{"image": img_tensor.squeeze()}]
    outputs = model(inputs)[0]  # Extract first output dictionary

    if "instances" not in outputs or len(outputs["instances"]) == 0:
        print("No objects detected in", image_path)
        return img, None

    instances = outputs["instances"]
    scores = instances.scores
    highest_class_idx = scores.argmax()
    scores[highest_class_idx].backward()

    # Compute Grad-CAM heatmap
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])  # Average over height & width
    feature_maps = feature_maps.squeeze(0)  # Remove batch dimension
    pooled_gradients = pooled_gradients.view(feature_maps.shape[0], 1, 1)

    cam = torch.sum(pooled_gradients * feature_maps, dim=0).cpu().detach().numpy()
    cam = np.maximum(cam, 0)  # Apply ReLU
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))  # Resize to match original image size

    # Overlay the heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * cam / cam.max()), cv2.COLORMAP_JET)
    overlayed_img = cv2.addWeighted(img, 0.5, heatmap, 0.5, 0)

    return img, overlayed_img  # Return original & heatmap image

# Run Grad-CAM on 8 Random Images & Display in Grid
test_folder = "/content/drive/My Drive/test"  # Change this to your test directory
selected_images = get_random_images(test_folder, num_images=8)

fig, axes = plt.subplots(2, 4, figsize=(20, 10))  # 2x4 grid for displaying images

for i, img_path in enumerate(selected_images):
    row, col = divmod(i, 4)

    original, heatmap = generate_grad_cam(img_path)
    if heatmap is None:
        continue  # Skip images with no detections

    # Display results
    axes[row, col].imshow(heatmap)
    axes[row, col].set_title(f"Image {i+1}")
    axes[row, col].axis("off")

# Show all Grad-CAM results
plt.show()


# PBE

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# Load the trained Detectron2 model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # Adjust based on your dataset
cfg.MODEL.WEIGHTS = "/content/drive/My Drive/model1_final.pth"  # Change to your trained model path
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

# Initialize model
model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)
model.eval()

# Initialize predictor for normal inference
predictor = DefaultPredictor(cfg)

# Function to get 8 random images from test folder
def get_random_images(folder, num_images=8):
    image_paths = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):  # Check valid image formats
                image_paths.append(os.path.join(root, file))

    if len(image_paths) == 0:
        raise ValueError("ERROR: No images found in test folder!")

    random.shuffle(image_paths)
    return image_paths[:num_images]

# Occlusion Sensitivity Function
def perturb_image(image, mask_size=50, stride=25):
    """
    Iteratively occludes different parts of the image and measures impact on predictions.
    Generates an importance heatmap showing regions most sensitive to occlusion.
    """
    h, w, _ = image.shape
    heatmap = np.zeros((h, w))

    # Run original prediction
    original_output = predictor(image)
    original_score = original_output["instances"].scores.max().item() if len(original_output["instances"]) > 0 else 0

    # Slide occlusion window across image
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            # Create occlusion mask
            occluded_image = image.copy()
            occluded_image[y:y+mask_size, x:x+mask_size] = 0  # Black-out occluded area

            # Run prediction on occluded image
            occluded_output = predictor(occluded_image)
            occluded_score = occluded_output["instances"].scores.max().item() if len(occluded_output["instances"]) > 0 else 0

            # Compute score drop (higher = more important)
            heatmap[y:y+mask_size, x:x+mask_size] = original_score - occluded_score

    # Normalize heatmap
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())
    return heatmap

# Run Perturbation-Based Explainability on 8 Random Images & Display in Grid
test_folder = "/content/drive/My Drive/test"  # Change this to your test directory
selected_images = get_random_images(test_folder, num_images=8)

fig, axes = plt.subplots(2, 4, figsize=(20, 10))  # 2x4 grid for displaying images

for i, img_path in enumerate(selected_images):
    row, col = divmod(i, 4)

    # Load image
    img = cv2.imread(img_path)
    if img is None:
        print(f"ERROR: Could not load {img_path}")
        continue
    img = cv2.resize(img, (512, 512))  # Resize for consistency
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Compute occlusion sensitivity heatmap
    heatmap = perturb_image(img)

    # Convert to color heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlayed_img = cv2.addWeighted(img, 0.5, heatmap, 0.5, 0)

    # Display results
    axes[row, col].imshow(overlayed_img)
    axes[row, col].set_title(f"Image {i+1}")
    axes[row, col].axis("off")

# Show all perturbation-based results
plt.show()
